In [4]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import warnings

import pandas_ta as ta
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_csv("train_기술적_분석.csv")
df.head()

,date,ticker,firm,volume,open,high,low,close,adjustTrue,SMA_10,...,BBP_5_2.0,ADX_14,DMP_14,DMN_14,EFI_13,KAMA_10_2_30,MFI_14,VTXP_14,VTXM_14,target
0,2022-03-23,A000020,동화약품,396150.0,13800.0,14100.0,13600.0,13650.0,1,13335.0,...,0.360653,20.440277,30.358104,13.516124,2.075979e+08,13031.963084,74.680545,1.109091,0.842424,12800.0
1,2022-03-24,A000020,동화약품,164839.0,13600.0,13700.0,13500.0,13600.0,1,13405.0,...,0.302562,21.452135,29.627900,14.393671,1.767636e+08,13056.980792,72.143290,1.086957,0.913043,12950.0
2,2022-03-25,A000020,동화약품,248995.0,13700.0,13950.0,13500.0,13900.0,1,13510.0,...,0.851733,22.711743,31.055802,13.600975,1.621829e+08,13116.285955,73.136564,1.078788,0.872727,13200.0
3,2022-03-28,A000020,동화약품,160036.0,13900.0,13900.0,13600.0,13750.0,1,13630.0,...,0.593659,23.881378,29.874583,13.083657,1.355845e+08,13175.382776,70.754830,1.092593,0.901235,13250.0
4,2022-03-29,A000020,동화약품,160334.0,13850.0,14000.0,13650.0,13750.0,1,13720.0,...,0.548564,25.101498,29.815148,12.486931,1.162153e+08,13215.806253,72.694644,1.118750,0.868750,13200.0


In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
from tqdm import tqdm
from joblib import Parallel, delayed

# XGBoost 하이퍼파라미터의 탐색 공간 정의
param_grid = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1]
}


# 특정 주식에 대해 Grid Search를 수행하는 함수
def grid_search_for_one_stock(ticker):
    # 특정 주식 데이터만 선택
    data = df[df['ticker'] == ticker]

    # 특성 및 타겟 정의
    X = data.drop('target', axis=1)
    y = data['target']

    # XGBoost 모델 생성
    model = XGBRegressor(objective='reg:squarederror', n_jobs=-1)

    # Grid Search 실행
    tscv = TimeSeriesSplit(n_splits=5)
    grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='neg_mean_squared_error')
    grid_search.fit(X, y)

    # 최적 하이퍼파라미터 반환
    return ticker, grid_search.best_params_

# 모든 주식에 대해 병렬로 Grid Search 실행
results = Parallel(n_jobs=-1)(delayed(grid_search_for_one_stock)(ticker) for ticker in df['ticker'].unique())

# 결과 출력
for ticker, best_params in results:
    print(f"Best parameters for {ticker}: {best_params}")
